In [ ]:
#Import package
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import SimpleRNN,Dense,LSTM
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.model_selection import GroupShuffleSplit
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score, matthews_corrcoef, precision_score, recall_score, f1_score

In [ ]:
import pandas as pd
dataframe = pd.read_csv("timesteps=3_NeuralNetwork.csv")
dataframe.head()

In [ ]:
#Split to training and testing data by proportion if testing = 0.2
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx,test_idx=next(gss.split(dataframe,groups=dataframe['stay_id']))


#Define x and y variables, and set the lables for the outcome, also reshape the x to fit the NN
train_data=dataframe.iloc[train_idx] # data
train_label=train_data['vent_duration'] # label
bins=[0,12,24,1000] # label
labels=[0,1,2] # label
train_label=pd.cut(train_label,bins=bins,labels=labels,include_lowest=True) # label
train_label.index=range(train_label.shape[0])
y_train=np.empty((train_label.shape[0]//3,1)) #label
for i in range(y_train.shape[0]):
    y_train[i]=train_label.loc[i*3]

x_train=train_data.drop(['timestep','stay_id','vent_duration'],axis=1) # data
scaler=preprocessing.StandardScaler()
x_train=scaler.fit_transform(x_train)
x_train=x_train.reshape(-1,3,12)

#
test_data=dataframe.iloc[test_idx] # data
test_label=test_data['vent_duration'] # label
bins=[0,12,24,1000] # label
labels=[0,1,2] # label
test_label=pd.cut(test_label,bins=bins,labels=labels,include_lowest=True) # label
test_label.index=range(test_label.shape[0])
y_test=np.empty((test_label.shape[0]//3,1)) #label
for j in range(y_test.shape[0]):
    y_test[j]=test_label.loc[j*3]

x_test=test_data.drop(['timestep','stay_id','vent_duration'],axis=1) # data
scaler=preprocessing.StandardScaler()
x_test=scaler.fit_transform(x_test)
x_test=x_test.reshape(-1,3,12)

In [ ]:
#FNN
model = Sequential()
model.add(Flatten(input_shape=(3, 12)))
model.add(Dense(64, activation='relu'))
#model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit(x_train,y_train, epochs=20,  batch_size = 20, verbose=0)

predictions=pd.DataFrame(model.predict(x_test))
y_pred=np.array(predictions.idxmax(axis=1))


In [ ]:
#Evaluation
def predict_evaluation(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    MCC = matthews_corrcoef(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    fscore = f1_score(y_test, y_pred, average='macro')
    result={'accuracy':accuracy,
            'MCC':MCC,
            'precision':precision,
            'recall':recall,
            'fscore':fscore}
    return result

result=predict_evaluation(y_pred,y_test)